In [1]:
import geopandas as gpd
import pandas as pd
import geowrangler.vector_zonal_stats as vzs
import geowrangler.raster_zonal_stats as rzs
from geowrangler import grids
import folium

import os
from pathlib import Path

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Processing Generated Isochrone contours

## Set-up parameters and directories

In [3]:
PROCESSED_DIR = Path("../../../data/03-processed/")
OUTPUT_DIR = Path("../../../data/04-output/")
GIS_DIR = Path("../../../data/05-gis/")

HOSPITALS_FPATH = Path("../../../data/02-raw/philippines_healthfacilities.gpkg")
ISOCHRONES_DIR = PROCESSED_DIR / "isochrones"
ADMIN_BOUNDS = Path("../../../data/01-admin-bounds/target_admin_bounds.shp")
WP_2020_RASTER = Path("../../../data/02-raw/worldpop/population_count/phl_ppp_2020.tif")

## Load health facilities points

In [4]:
doh_gdf = gpd.read_file(HOSPITALS_FPATH, driver="GPKG")
doh_gdf.head(2)

,id,facilityco,healthfaci,typeofheal,barangay,municipali,province,region,status,address,style,geometry
0,1.0,DOH000000000002277,Calvario Barangay Health Station,Barangay Health Station,Calvario,City Of Isabela,City Of Isabela (not A Province),REGION IX (ZAMBOANGA PENINSULA),None,None,Barangay Health Station,POINT (121.98987 6.65182)
1,2.0,DOH000000000010319,Cabunbata Barangay Health Station,Barangay Health Station,Cabunbata,City Of Isabela,City Of Isabela (not A Province),REGION IX (ZAMBOANGA PENINSULA),None,None,Barangay Health Station,POINT (121.96630 6.67152)


In [5]:
# filter to target cities
target_cities = [
    "Navotas",
    "Mandaluyong",
    "Muntinlupa",
    "Dagupan City",
    "Palayan City",
    "Legazpi City",
    "Iloilo City",
    "Mandaue City",
    "Tacloban City",
    "Zamboanga City",
    "Cagayan de Oro City",
    "Davao City",
]
filtered_doh = doh_gdf[
    doh_gdf["municipali"].str.contains("|".join(target_cities), case=False)
]
hospitals_gdf = filtered_doh[filtered_doh["typeofheal"] == "Hospital"]

In [6]:
hospitals_gdf = hospitals_gdf.to_crs("epsg:4326")

In [7]:
hospitals_gdf = hospitals_gdf.rename(columns={"style": "type"})

## Load Isochrones

In [8]:
iso_files = os.listdir(ISOCHRONES_DIR)
print(iso_files)

['iso_hospital_drivetraffic_9, 11, 12.geojson', 'iso_hospital_drivetraffic_15.geojson', 'iso_hospital_drivetraffic_13, 14, 16.geojson', 'iso_hospital_drivetraffic_21, 22, 23.geojson', 'iso_hospital_drivetraffic_24, 26, 27.geojson', 'iso_hospital_drivetraffic_25.geojson', 'iso_hospital_drivetraffic_10.geojson', 'iso_hospital_drivetraffic_20.geojson', 'iso_hospital_drivetraffic_28, 29.geojson', 'iso_hospital_drivetraffic_17, 18, 19.geojson', 'iso_hospital_drivetraffic_5.geojson', 'iso_hospital_drivetraffic_6, 7, 8.geojson', 'iso_hospital_drivetraffic_30.geojson']


In [9]:
iso_gdf_list = []

for file in iso_files:
    gdf = gpd.read_file(ISOCHRONES_DIR / file, driver="GeoJSON")
    iso_gdf_list.append(gdf)

In [10]:
concat_iso = gpd.GeoDataFrame(pd.concat(iso_gdf_list, ignore_index=True))
concat_iso

,contour,uid,geometry
0,12,28.0,"POLYGON ((122.07181 6.93314, 122.07081 6.93197..."
1,11,28.0,"POLYGON ((122.07381 6.93123, 122.07281 6.93163..."
2,9,28.0,"POLYGON ((122.07481 6.92608, 122.07395 6.92382..."
3,12,32.0,"POLYGON ((122.07054 6.93523, 122.06935 6.93252..."
4,11,32.0,"POLYGON ((122.07554 6.93242, 122.07354 6.93247..."
...,...,...,...
2855,30,20201.0,"POLYGON ((125.64783 7.19857, 125.64609 7.19608..."
2856,30,20202.0,"POLYGON ((125.57941 7.13767, 125.57741 7.13673..."
2857,30,20825.0,"POLYGON ((125.64870 7.24679, 125.64662 7.24529..."
2858,30,20830.0,"POLYGON ((125.55393 7.23796, 125.55250 7.23467..."


In [11]:
concat_iso = concat_iso.set_crs("epsg:4326")

## Load AOI

In [12]:
aoi = gpd.read_file(ADMIN_BOUNDS)
aoi.head(3)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060..."


## Generate grids

In [13]:
bing_tile_grid_generator = grids.BingTileGridGenerator(18)

In [14]:
%%time
bing_tile_gdf = bing_tile_grid_generator.generate_grid(aoi)

CPU times: user 7min 57s, sys: 134 ms, total: 7min 57s
Wall time: 7min 58s


In [15]:
bing_tile_gdf.shape

(223316, 2)

## Convert contours to grids

In [17]:
%%time
iso_grids = vzs.create_zonal_stats(
    bing_tile_gdf,
    concat_iso,
    aggregations=[
        {"func": "min", "column": "contour", "output": "hospital_travel_time"}
    ],
)

CPU times: user 1min 1s, sys: 560 ms, total: 1min 1s
Wall time: 1min 1s


In [18]:
iso_grids.head(3)

,quadkey,geometry,hospital_travel_time
0,132303122020330010,"POLYGON ((121.00616 14.36950, 121.00616 14.370...",15.0
1,132303122020330012,"POLYGON ((121.00616 14.36817, 121.00616 14.369...",15.0
2,132303122020330030,"POLYGON ((121.00616 14.36684, 121.00616 14.368...",17.0


## Add population

In [20]:
%%time
iso_grids_pop = rzs.create_raster_zonal_stats(
    iso_grids,
    WP_2020_RASTER,
    aggregation=dict(func="sum", column="population", output="population_count"),
    extra_args=dict(nodata=-99999),
)

CPU times: user 5min 48s, sys: 13.2 s, total: 6min 2s
Wall time: 6min 2s


In [21]:
iso_grids_pop.head(5)

,quadkey,geometry,hospital_travel_time,population_count
0,132303122020330010,"POLYGON ((121.00616 14.36950, 121.00616 14.370...",15.0,59.810890
1,132303122020330012,"POLYGON ((121.00616 14.36817, 121.00616 14.369...",15.0,73.748970
2,132303122020330030,"POLYGON ((121.00616 14.36684, 121.00616 14.368...",17.0,38.532951
3,132303122020330032,"POLYGON ((121.00616 14.36551, 121.00616 14.366...",17.0,76.930756
4,132303122020330210,"POLYGON ((121.00616 14.36418, 121.00616 14.365...",18.0,39.472054


In [22]:
iso_grids_pop.to_file(
    OUTPUT_DIR / "nearest_hospitals_iso_grids.geojson", driver="GeoJSON"
)

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## Create visualizations

In [23]:
# Set map center
gdf_centroid_lat, gdf_centroid_lon = (
    aoi.geometry.centroid.y.mean(),
    aoi.geometry.centroid.x.mean(),
)

/tmp/ipykernel_271/2846051968.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi.geometry.centroid.y.mean(),
/tmp/ipykernel_271/2846051968.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi.geometry.centroid.x.mean(),


In [31]:
map = folium.Map(
    location=[gdf_centroid_lat, gdf_centroid_lon],
    zoom_start=7,
    control_scale=True,
    tiles="cartodb positron",
)

iso_grids_pop.explore(m=map, column="hospital_travel_time", cmap="Blues_r")
aoi.explore(m=map, style_kwds={"weight": 3, "color": "cyan", "fill": False})
hospitals_gdf.explore(m=map, color="yellow", marker_kwds={"radius": 4})
map.save(GIS_DIR / "lacuna_hospitals_iso_grids_wpoints_pop_1min.html")